# Imports

In [1]:
from collections import defaultdict
from pathlib2 import Path
from riotwatcher import LolWatcher, ApiError


import arrow
import json
import numpy as np
import pandas as pd

# Functions

In [2]:
  
def api_key(credentials):
    """Reads in the development API key from a file and checks if it is viable 
    
    Input is a JSON file 

    """
    with open(api_key_loc, "r") as json_data:
            credentials = json.load(json_data)
    return api_key

# TODO: Currently broken update when riot watcher is able to use lol_status_v4
#     # while True:
#         with open(api_key_loc, "r") as json_data:
#             credentials = json.load(json_data)
#             api_key = credentials["dev_api_key"]
#             lol_watcher = LolWatcher(credentials["dev_api_key"])
#             try:
#                 # Use the current API key to check server status
#                 server_status = lol_watcher.lol_status.shard_data("euw1")
#                 # break if key is functional
#                 break
#             except ApiError as error:
#                 # If the current API key does not work input new one
#                 if error.response.status_code == 403:
#                     new_api_key = input("API key is incorrect, enter correct key here.")
#                     credentials["dev_api_key"] = new_api_key
#                     # Replace the old API key
#                     with open(api_key_loc, "w") as json_data:
#                         json.dump(credentials, json_data)
#     return api_key

In [3]:
def kda(kill, death, assist):
    #somehow does not work
    """Takes the kills, deaths, and assists of a player and calculates the KDA ratio order is kills, deaths, assists 
    
    Input is string

    """
    score = round((kills+assists) / max(1, death), 3)

    return score

# Main

In [8]:
# Folder locations

# Project folders
project_folder = Path.cwd().parent

# Raw data storage
data_folder = project_folder / "data"

# Out folder
out_folder = project_folder / "out"

In [9]:
# Set API key
api_key_loc = data_folder / "dev_api_key.json"
# lol_watcher = LolWatcher(api_key(api_key_loc))

with open(api_key_loc, "r") as json_data:
            credentials = json.load(json_data)
            api_key = credentials["dev_api_key"]
            lol_watcher = LolWatcher(credentials["dev_api_key"])

In [10]:
# Set user request parameters

# Set region
region = "euw1"

# Set summonder name 
protagonist = "alnas" # not case sensitive

### User info

In [11]:
# retrieve info using region and summoner name
user = lol_watcher.summoner.by_name(region, protagonist)
print(user)

{'id': 'L2OqMzNqsCelFfcFlPx5iQWRB7dKLDdf9xGgO_W1dNau3EI', 'accountId': 'fYybzHGf_HSjunX7VEs-9jXIEvVLk-eQhAyhq_VzaXWi2w', 'puuid': 'tKPszy2ygrLuiS7AYcfh4pjDzc3jQO2xOIjAf4YKq8XjytTznNaQc88wPYMmJkONT1JjtkioHEfaMg', 'name': 'Alnas', 'profileIconId': 3382, 'revisionDate': 1612224300000, 'summonerLevel': 107}


In [12]:
# Check rank stats using user id
ranked_stats = lol_watcher.league.by_summoner(region, user["id"])
print(ranked_stats)

[{'leagueId': 'f45c0593-8eab-41ca-a45e-987f2cff8cca', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'GOLD', 'rank': 'IV', 'summonerId': 'L2OqMzNqsCelFfcFlPx5iQWRB7dKLDdf9xGgO_W1dNau3EI', 'summonerName': 'Alnas', 'leaguePoints': 50, 'wins': 16, 'losses': 18, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}]


### Match info

In [14]:
# set match request parameters

# Set queue
queue = 420

# Set season ????

In [26]:
# Retrieve match id from a single match, ranked queue based on time
# (max is last 100 matches https://developer.riotgames.com/apis#match-v4) but time range is 1 week 


matches = lol_watcher.match.matchlist_by_account(region=region,
                                                 encrypted_account_id=user["accountId"],
                                                 queue=queue,
                                                 begin_index=0,
                                                 end_index=100)
#TODO: make this work with time

                                                 
print(matches.keys())
print(matches["matches"][0])
# print(matches["startIndex"])
# print(matches["endIndex"])
# print(matches["totalGames"]) 


dict_keys(['matches', 'startIndex', 'endIndex', 'totalGames'])
{'platformId': 'EUW1', 'gameId': 5064394827, 'champion': 412, 'queue': 420, 'season': 13, 'timestamp': 1612040868387, 'role': 'SOLO', 'lane': 'BOTTOM'}


In [27]:
# Create a dictionary to store all the results
match_info = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))

# Get match details of last 10 matches
for match in matches["matches"][:10]:
        # Store filtered information pertinent to the match itself
        game_id = match["gameId"]
        queue_type = match["queue"]
        match_info[game_id]["game_info"]["game_type"] = queue_type
        match_info[game_id]["game_info"]["begin_time"] = None
        match_info[game_id]["game_info"]["end_time"] = None
        # TODO: add time 


        # Store filtered information pertinent to participants
        match_detail = lol_watcher.match.by_id(region, game_id)
        protagonist_blue = None
        # Collect participant ID and summoner name
        for participant in match_detail["participantIdentities"]:
                part_id = participant["participantId"]
                summoner_name = participant["player"]["summonerName"]
                # Storing summoner name using participant ID as key
                match_info[game_id]["player_data"][part_id]["participant_id"] = part_id 
                match_info[game_id]["player_data"][part_id]["name"] = summoner_name
                if summoner_name.lower() == protagonist.lower():
                        protagonist_blue = True if part_id <=5 else False

        # Collect kills, deaths, assists and win
        for participant in match_detail["participants"]:
                part_id = participant["stats"]["participantId"]
                kills = participant["stats"]["kills"] 
                deaths = participant["stats"]["deaths"]
                assists = participant["stats"]["assists"]
                kda = round((kills+assists)/ max(1, deaths), 3)
                win = participant["stats"]["win"]

                # Store kills, deaths, assists, kda and win
                match_info[game_id]["player_data"][part_id]["kills"] = kills
                match_info[game_id]["player_data"][part_id]["deaths"] = deaths
                match_info[game_id]["player_data"][part_id]["assists"] = assists
                match_info[game_id]["player_data"][part_id]["kda"] = kda
                match_info[game_id]["player_data"][part_id]["win"] = win

                # Assign participants as ally or enemy of the protagonist                   
                match_info[game_id]["player_data"][part_id]["allied"] = bool(part_id <=5) == protagonist_blue

print("done")
# TODO: print how long it took

done


In [28]:
# Output for for easy reading
for i in match_info.keys():
        print("this is match", i)
        print(match_info[i].keys())
        print(match_info[i]["game_info"].keys())
        print("")

        print("match info")
        for y in match_info[i]["game_info"]:
                print(match_info[i]["game_info"][y])

        print("")
        print("player info")
        for player in match_info[i]["player_data"]:
                print(match_info[i]["player_data"][player])

        print("\n------------------------\n")

this is match 5064394827
dict_keys(['game_info', 'player_data'])
dict_keys(['game_type', 'begin_time', 'end_time'])

match info
420
None
None

player info
{'participant_id': 1, 'name': 'HeroicBrother', 'kills': 6, 'deaths': 11, 'assists': 6, 'kda': 1.091, 'win': False, 'allied': True}
{'participant_id': 2, 'name': 'viscer0', 'kills': 2, 'deaths': 5, 'assists': 3, 'kda': 1.0, 'win': False, 'allied': True}
{'participant_id': 3, 'name': 'Alnas', 'kills': 0, 'deaths': 5, 'assists': 6, 'kda': 1.2, 'win': False, 'allied': True}
{'participant_id': 4, 'name': 'Likeabossbart', 'kills': 3, 'deaths': 8, 'assists': 3, 'kda': 0.75, 'win': False, 'allied': True}
{'participant_id': 5, 'name': 'Lux4200', 'kills': 4, 'deaths': 7, 'assists': 1, 'kda': 0.714, 'win': False, 'allied': True}
{'participant_id': 6, 'name': 'HomeMadeCookies', 'kills': 6, 'deaths': 3, 'assists': 7, 'kda': 4.333, 'win': True, 'allied': False}
{'participant_id': 7, 'name': 'CoolFroggy', 'kills': 1, 'deaths': 1, 'assists': 19, 'kd

In [35]:



# print(player["participant_id"])

print(match_detail.keys()) # also contains game duration this might be interresting
# print(match_detail["participantIdentities"])


# print(match_detail["teams"])

# for i in match_detail["teams"]:
#         print(i)


# steps to take.
# get teams. check
# assign players to teams
#
# get a form like this ID, name, win of protagonist
# group all the ones that have win together and lose together based on protagonist
# Add player names to table of interrest
# add win or lose status to players 



# for player in match_detail["participants"]:
#         print(player)
#         break
# #     # print(player.keys()) #'participantId', 'teamId', 'championId', 'spell1Id', 'spell2Id', 'stats', 'timeline'])
# #     # print(player["participantId"]) # is also found in stats so this one can be ignored
# #     # print(player["teamId"]) # 100 is blue 200 is red prolly but this is useless
# #     # print(player["championId"]) # which champ they are playing maaybe needed to link back to player
#     print(player["stats"].keys()) # is a dictionary of player stats during the game
#     break
#     # print(player["timeline"])
# {teams:allied{players:{sumname{},kills{}}},enemy{}}



print(match_detail['gameCreation'])
print(match_detail["gameDuration"])
# print(part_id_sum_name_dict)



dict_keys(['gameId', 'platformId', 'gameCreation', 'gameDuration', 'queueId', 'mapId', 'seasonId', 'gameVersion', 'gameMode', 'gameType', 'teams', 'participants', 'participantIdentities'])
1611592052124
1881


In [ ]:


# participants = []
# for row in match_detail['participants']:
#     participants_row = {}
#     participants_row['champion'] = row['championId']
#     # participants_row['spell1'] = row['spell1Id']
#     # participants_row['spell2'] = row['spell2Id']
#     participants_row['win'] = row['stats']['win']
#     participants_row['kills'] = row['stats']['kills']
#     participants_row['deaths'] = row['stats']['deaths']
#     participants_row['assists'] = row['stats']['assists']
#     participants_row['KDA'] = kda(row['stats']['kills'], row['stats']['deaths'], row['stats']['assists'])
#     participants_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
#     participants_row['goldEarned'] = row['stats']['goldEarned']
#     participants_row['champLevel'] = row['stats']['champLevel']
#     participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
#     # participants_row['item0'] = row['stats']['item0']
#     # participants_row['item1'] = row['stats']['item1']
#     participants.append(participants_row)
# df = pd.DataFrame(participants)
# df

### Patch an season info

In [62]:
# example of how to Read time
# 2018-01-24T03:00:00-08:00  (for patch 8.2) --> corresponds to 3am on January 24th, 2018 in Pacific Time

with open(data_folder / "patches.json") as f:
    data = json.load(f)

patch = data["patches"][-1]  # most recent
utc_timestamp = patch["start"]
north_america_timestamp = utc_timestamp + data["shifts"]["NA1"]
print(north_america_timestamp)
dt = arrow.get(north_america_timestamp)
print(dt.to("US/Pacific"))



1607515200
2020-12-09T04:00:00-08:00


In [ ]:
# example end script for wayyy later

try:
    response = lol_watcher.summoner.by_name(region, summoner_name)
    print(response)
except ApiError as error:
    if error.response.status_code == 429:
        print('Request limit reached retry in {} seconds.'.format(err.headers['Retry-After']))
    elif error.response.status_code == 404: 
        print("Found no match")